# Import Libraries

In [23]:

from pyspark.ml.recommendation import ALS
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.types import StructType,StructField,IntegerType


# Constants : Modify as and when required!

In [24]:
# file paths
business_file="Sample_Datasets/montreal_business/part-00000-b5f251d0-79e6-47a8-a405-042eb7b7894e-c000.snappy.parquet"
reviews_file="Sample_Datasets/montreal_reviews/part-00000-f0e4463e-0ac9-402e-b995-734cbefc958e-c000.snappy.parquet"
users_file="Sample_Datasets/montreal_users/part-00000-a7d49d78-89a7-478f-a577-0efe02dca047-c000.snappy.parquet"


# Initialize spark session

In [25]:
app_name="Collaborative filtering for restaurant recommendation"

def init_spark():
    spark = SparkSession \
        .builder \
        .appName(app_name) \
        .getOrCreate()
    return spark


# Load Dataset in Apache Spark

In [26]:
spark=init_spark()
business_df = spark.read.parquet(business_file)
reviews_df=spark.read.parquet(reviews_file)
users_df=spark.read.parquet(users_file)

## Selecting required features

In our project, we are only concerned with a subset of columns from the dataset, specifically those that are relevant to our goal of recommending restaurants in Montreal. Therefore, we extract the necessary features from the business_df table, including the id, name, stars, category. 
Similarly, we filter the reviews_df=spark.read.parquet(reviews_file)
 table to include only reviews for the selected restaurants by performing an inner join with business_df.

In [27]:
business_df = business_df.select("business_id","name", "stars", 
                                 "review_count", "address", "city", "state", "postal_code", "longitude", 
                                 "categories", "latitude").withColumnRenamed("stars", "stars_restaurant")
reviews_df = reviews_df.join(business_df, on='business_id', how='inner')

## Preparing Data for ALS: Convert String to index
Prior to initiating the modeling process, it is essential to transform all the relevant columns to integer type for compatibility with the ALS model from pyspark. The columns requiring conversion are the business_id and user_id. We accomplish this by leveraging the StringIndexer function, which we imported from pyspark.ml.feature.

In [28]:
indexer = [StringIndexer(inputCol=column, outputCol=column+"_index") for column in ['business_id', 'user_id']]
pipeline = Pipeline(stages=indexer)
transformed = pipeline.fit(reviews_df).transform(reviews_df)
transformed.select(['business_id', 'user_id','business_id_index', 'user_id_index','stars','categories'])

DataFrame[business_id: string, user_id: string, business_id_index: double, user_id_index: double, stars: double, categories: string]

## Spliting the dataset into training and testing subsets 

In [29]:
(training, test) = transformed.randomSplit([0.8, 0.2])

## Create ALS model
The Apache Spark library provides various parameters for the ALS (Alternating Least Squares) algorithm, including:

- numBlocks*: the number of blocks used to partition users and items, allowing for parallelized computation (default value: 10).
- rank: the number of latent factors used in the model (default value: 10).
- maxIter: the maximum number of iterations to run (default value: 10).
- regParam: the regularization parameter used in ALS (default value: 1.0).
- implicitPrefs: a boolean value that indicates whether to use the explicit feedback ALS variant or the one adapted for implicit feedback data (default value: false, which means using explicit feedback).
- alpha: a parameter that applies to the implicit feedback variant of ALS, determining the baseline confidence in preference observations (default value: 1.0).
- nonnegative: a boolean value that specifies whether to use nonnegative constraints for least squares (default value: false).

In [30]:
als=ALS(maxIter=5,
        regParam=0.09,
        rank=20,
        userCol="user_id_index",
        itemCol="business_id_index",
        ratingCol="stars",
        coldStartStrategy="drop",
        nonnegative=True)

model=als.fit(training)

## Evaluate RMSE

In [31]:
evaluator=RegressionEvaluator(metricName="rmse",labelCol="stars",predictionCol="prediction")
predictions=model.transform(test)
rmse=evaluator.evaluate(predictions)
print("RMSE="+str(rmse))

RMSE=1.44135245214684


## Tuning Hyper Parameters

In [12]:
als = ALS(maxIter=20,regParam=0.09,rank=20,userCol="user_id_index",itemCol="business_id_index",ratingCol="stars",coldStartStrategy="drop",nonnegative=True)
model = als.fit(training)

## Re-evaluating the RMSE

In [13]:
evaluator=RegressionEvaluator(metricName="rmse",labelCol="stars",predictionCol="prediction")
predictions=model.transform(test)
rmse=evaluator.evaluate(predictions)
print("RMSE="+str(rmse))

RMSE=1.3138577287018127


In [14]:
als = ALS(maxIter=20,regParam=0.09,rank=25,userCol="user_id_index",itemCol="business_id_index",ratingCol="stars",coldStartStrategy="drop",nonnegative=True)
model = als.fit(training)

In [15]:
evaluator=RegressionEvaluator(metricName="rmse",labelCol="stars",predictionCol="prediction")
predictions=model.transform(test)
rmse=evaluator.evaluate(predictions)
print("RMSE="+str(rmse))

RMSE=1.3108924547504515


In [16]:
values=[0.08,0.09,0.1,0.15,0.2,0.25,0.3,0.4,0.45,0.5,0.5,0.6,0.7,0.8,0.9]
for rm in values:
    als = ALS(maxIter=20,regParam=rm,rank=25,userCol="user_id_index",itemCol="business_id_index",ratingCol="stars",coldStartStrategy="drop",nonnegative=True)
    model = als.fit(training)
    evaluator=RegressionEvaluator(metricName="rmse",labelCol="stars",predictionCol="prediction")
    predictions=model.transform(test)
    rmse=evaluator.evaluate(predictions)
    print("RMSE="+str(rmse))
    print(model.userFactors.count())
    print(model.itemFactors.count())
    

RMSE=1.3183231057620133
42322
4169
RMSE=1.3108924547504517
42322
4169
RMSE=1.3035536605930647
42322
4169
RMSE=1.2707102449526586
42322
4169
RMSE=1.247503770638856
42322
4169
RMSE=1.233860236681604
42322
4169
RMSE=1.226375515158736
42322
4169
RMSE=1.2246239501304896
42322
4169
RMSE=1.2290809657047284
42322
4169
RMSE=1.2368554283162059
42322
4169
RMSE=1.2368554283162063
42322
4169
RMSE=1.2626710635715015
42322
4169
RMSE=1.300965824561188
42322
4169
RMSE=1.3487676080836872
42322
4169
RMSE=1.4033817268967461
42322
4169


In [ ]:
#we are uisng 0.4 as the regularizatin parameter as it yields lowest RMSE

In [ ]:
als = ALS(maxIter=30,regParam=0.4,rank=25,userCol="user_id_index",itemCol="business_id_index",ratingCol="stars",coldStartStrategy="drop",nonnegative=True)
model = als.fit(training)
evaluator=RegressionEvaluator(metricName="rmse",labelCol="stars",predictionCol="prediction")
predictions=model.transform(test)
rmse=evaluator.evaluate(predictions)
print("RMSE="+str(rmse))

# Find Best Recommendations

In [18]:

test = model.recommendForAllUsers(20).filter(col('user_id_index')==30).select("recommendations").collect()
topRestaurants = []
for item in test[0][0]:        
    topRestaurants.append(item.business_id_index)
    
schema = StructType([StructField("business_id_index",IntegerType(),True)])
restaurants = spark.createDataFrame(topRestaurants,IntegerType()).toDF("business_id_index")


transformed\
.select(['business_id', 'user_id', 'stars', 'categories'])\
.filter(col('user_id_index')==30)\
.show()

restaurants\
.join(transformed, on = 'business_id_index', how = 'inner')\
.select(['business_id', 'stars', 'categories', 'name'])\
.drop_duplicates(subset=['name'])\
.show()

predictions.select(['user_id', 'business_id', 'stars', 'prediction']).show()


+--------------------+--------------------+-----+--------------------+
|         business_id|             user_id|stars|          categories|
+--------------------+--------------------+-----+--------------------+
|ubLn_FrFygzcbhXTD...|0gZ8E5tBWTEtGEZDu...|  5.0|Restaurants, Cafe...|
|zrnP9HqoF-RI9jqoW...|0gZ8E5tBWTEtGEZDu...|  5.0|Coffee & Tea, Res...|
|yU3p0tEBtGuZLTcsB...|0gZ8E5tBWTEtGEZDu...|  5.0|Restaurants, Brea...|
|tmWp4Rtr_cm7nCh2u...|0gZ8E5tBWTEtGEZDu...|  5.0|Breakfast & Brunc...|
|vaURDGRo19cMB3Fy9...|0gZ8E5tBWTEtGEZDu...|  5.0|Restaurants, Coff...|
|58APdML-PG_OD4El2...|0gZ8E5tBWTEtGEZDu...|  2.0|Delicatessen, Foo...|
|ZL9sk7Imc42BNRhcj...|0gZ8E5tBWTEtGEZDu...|  4.0|Chinese, Restaura...|
|lpnuObNKbkH8usnUS...|0gZ8E5tBWTEtGEZDu...|  4.0|Cafes, Coffee & T...|
|q91nljuSDFl0IYRey...|0gZ8E5tBWTEtGEZDu...|  3.0|Vegan, Vegetarian...|
|8cPMjIwTnrVm3F2wL...|0gZ8E5tBWTEtGEZDu...|  4.0|Vegan, Cafes, Veg...|
|xlMgaPOpd_99SFTuO...|0gZ8E5tBWTEtGEZDu...|  4.0|Game Meat, Americ...|
|vYbpo